# EDA: CMS DoubleMuon Run2016G (NanoAOD) — Dataset Understanding

**Goal:** Parse the `index.json_*` files to obtain a file list, then stream a small sample of NanoAOD ROOT files and perform EDA:
- dataset manifest (how many files, what paths)
- NanoAOD structure (trees, branches, collections)
- event-level EDA (run/lumi, PV, MET)
- muon & dimuon EDA (multiplicities, kinematics, invariant mass peaks)
- trigger (HLT_*) inventory and basic rates

We will keep everything reproducible and “CERN-reviewable”:
- deterministic sampling (seed)
- minimal assumptions about the index file format (we detect and adapt)
- no full downloads required; we read only a limited number of events for EDA


In [1]:
# Cell 1 — Install (safe to run even if already installed)
# If you only have local ROOT files (not remote), XRootD may not be necessary.
!pip -q install "uproot>=5" awkward vector rich tqdm pandas pyarrow fastparquet
try:
    import XRootD  # noqa: F401
except Exception:
    # Needed only for root:// streaming. If this fails, you can still analyze local files.
    !pip -q install XRootD


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.8/393.8 kB 9.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.6/919.6 kB 25.7 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.7/656.7 kB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 58.0 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for XRootD (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for XRootD
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml b

In [2]:
# Cell 3 — Locate index files
index_files = []
for x in INDEX_SUFFIXES:
    # Match anything that ends with index.json_x
    # Example: CMS_Run2016G_..._file_index.json_0
    pattern = re.compile(rf"index\.json_{x}$")
    for p in BASE_PATH.rglob("*"):
        if p.is_file() and pattern.search(p.name):
            index_files.append(p)

index_files = sorted(index_files)
index_files, len(index_files)


NameError: name 'INDEX_SUFFIXES' is not defined

In [ ]:
# Cell 4 — Helper: robust index parser
# We don't assume a single JSON schema; we try:
# 1) JSON list/dict
# 2) JSON Lines
# 3) plain text (one file per line)
#
# We extract anything that looks like a ROOT file path/URL (endswith .root).

def extract_root_paths(obj):
    """Recursively extract .root strings from nested dict/list."""
    out = []
    if isinstance(obj, str):
        if obj.strip().endswith(".root") or ".root?" in obj:
            out.append(obj.strip())
    elif isinstance(obj, dict):
        for v in obj.values():
            out.extend(extract_root_paths(v))
    elif isinstance(obj, list):
        for it in obj:
            out.extend(extract_root_paths(it))
    return out

def parse_index_file(path: Path, max_lines=None):
    # Peek first non-empty char to guess format
    with path.open("rb") as f:
        head = f.read(4096)
    head_text = head.decode("utf-8", errors="ignore").lstrip()
    first_char = head_text[:1]

    roots = []

    # Case A: JSON container
    if first_char in ["{", "["]:
        try:
            with path.open("r", encoding="utf-8", errors="ignore") as f:
                data = json.load(f)
            roots = extract_root_paths(data)
            return roots, "json"
        except Exception:
            pass

    # Case B: JSON lines or plain text lines
    roots = []
    with path.open("r", encoding="utf-8", errors="ignore") as f:
        for i, line in enumerate(f):
            if max_lines is not None and i >= max_lines:
                break
            s = line.strip()
            if not s:
                continue
            # Try JSON line
            if s[0] in ["{", "["]:
                try:
                    data = json.loads(s)
                    roots.extend(extract_root_paths(data))
                    continue
                except Exception:
                    pass
            # Fallback: treat as plain text and extract any token that looks like a ROOT file
            # e.g. "root://.../file.root" or "/eos/.../file.root"
            m = re.findall(r"(\S+?\.root(?:\?\S+)?)", s)
            roots.extend(m)

    kind = "jsonl_or_text"
    return roots, kind


In [ ]:
# Cell 5 — Build a file manifest from all index files
all_roots = []
parse_meta = []

for p in index_files:
    roots, kind = parse_index_file(p)
    parse_meta.append({"index_file": str(p), "parse_kind": kind, "n_root_paths": len(roots)})
    all_roots.extend(roots)

meta_df = pd.DataFrame(parse_meta)
meta_df


In [ ]:
# Cell 6 — Clean & de-duplicate ROOT paths
def normalize_path(s):
    s = s.strip().strip('"').strip("'")
    # Some indices include EOS paths without protocol. If you already have root:// paths, keep them.
    # If it's a /eos/opendata/... path, you can turn it into root://eospublic.cern.ch//eos/opendata/...
    if s.startswith("/eos/opendata/"):
        return "root://eospublic.cern.ch//" + s.lstrip("/")
    return s

roots = [normalize_path(r) for r in all_roots]
roots = [r for r in roots if r.endswith(".root") or ".root?" in r]
roots = pd.Series(roots).drop_duplicates().tolist()

len(roots), roots[:5]


In [ ]:
# Cell 7 — Save manifest (useful for debugging & reproducibility)
manifest = pd.DataFrame({"root_file": roots})
manifest_path = Path("/kaggle/working/root_files_manifest.csv")
manifest.to_csv(manifest_path, index=False)
manifest_path, manifest.head()


## Quick sanity checks

We now:
1. pick a small deterministic sample of ROOT files
2. open one file to inspect available TTrees (usually: `Events`, sometimes `Runs`, `LuminosityBlocks`)
3. inspect branches to decide which features are safe to use in EDA


In [ ]:
# Cell 8 — Deterministic sampling of files for EDA
N_FILES_FOR_EDA = 3   # start tiny
sample_files = roots[:N_FILES_FOR_EDA]  # deterministic; you can also do random.sample(roots, N_FILES_FOR_EDA)

sample_files


In [ ]:
# Cell 9 — Inspect a single file structure
test_file = sample_files[0]
f = uproot.open(test_file)

f.keys()


In [ ]:
# Cell 10 — Inspect Events tree & branch inventory
events = f["Events"]
branch_names = events.keys()

len(branch_names), branch_names[:30]


In [ ]:
# Cell 11 — Helper to find branches by prefix (Muon_, HLT_, PV_, MET_, etc.)
def find_branches(prefix, names):
    return sorted([n for n in names if n.startswith(prefix)])

for pref in ["Muon_", "HLT_", "PV_", "MET_", "Jet_", "Electron_", "run", "luminosityBlock", "event"]:
    hits = [pref] if pref in branch_names else find_branches(pref, branch_names)
    print(pref, "->", len(hits))
    print(hits[:20], "\n")


## Load a small event sample (EDA slice)

We will read a limited number of entries so EDA runs fast.  
If you see memory errors, reduce `ENTRY_STOP` or restrict to fewer branches.


In [ ]:
# Cell 12 — Choose EDA branches (keep this minimal at first)
# NanoAOD branch names are flat, like Muon_pt, Muon_eta (jagged arrays).
BASE_BRANCHES = ["run", "luminosityBlock", "event", "PV_npvs", "MET_pt", "MET_phi"]
MUON_BRANCHES = [
    "Muon_pt", "Muon_eta", "Muon_phi", "Muon_charge",
    "Muon_tightId", "Muon_mediumId",
    "Muon_pfRelIso03_all", "Muon_dxy", "Muon_dz"
]
# Trigger inventory can be large; we’ll read a filtered set later.

wanted = [b for b in (BASE_BRANCHES + MUON_BRANCHES) if b in branch_names]
missing = sorted(set(BASE_BRANCHES + MUON_BRANCHES) - set(wanted))

wanted, missing[:20], len(missing)


In [ ]:
# Cell 13 — Read arrays from one file (small slice)
ENTRY_STOP = 200_000  # adjust as needed

arr = events.arrays(wanted, entry_stop=ENTRY_STOP, library="ak")
{ k: (arr[k].type if k in arr.fields else None) for k in arr.fields }


In [ ]:
# Cell 14 — Event-level EDA tables
df_evt = pd.DataFrame({
    "run": ak.to_numpy(arr["run"]),
    "lumi": ak.to_numpy(arr["luminosityBlock"]),
    "event": ak.to_numpy(arr["event"]),
    "PV_npvs": ak.to_numpy(arr["PV_npvs"]) if "PV_npvs" in arr.fields else np.nan,
    "MET_pt": ak.to_numpy(arr["MET_pt"]) if "MET_pt" in arr.fields else np.nan,
})

df_evt.describe(include="all")


In [ ]:
# Cell 15 — Run/lumi coverage in the slice
df_evt[["run", "lumi"]].value_counts().head(10)


In [ ]:
# Cell 16 — Muon multiplicity distribution
mu_n = ak.num(arr["Muon_pt"])
mu_n_pd = pd.Series(ak.to_numpy(mu_n), name="nMuon")

mu_n_pd.value_counts().sort_index().head(20), mu_n_pd.describe()


In [ ]:
# Cell 17 — Basic muon kinematics (flatten all muons in the slice)
mu_pt = ak.flatten(arr["Muon_pt"])
mu_eta = ak.flatten(arr["Muon_eta"])
mu_iso = ak.flatten(arr["Muon_pfRelIso03_all"]) if "Muon_pfRelIso03_all" in arr.fields else None

mu_summary = pd.DataFrame({
    "Muon_pt": ak.to_numpy(mu_pt),
    "Muon_eta": ak.to_numpy(mu_eta),
    "Muon_pfRelIso03_all": ak.to_numpy(mu_iso) if mu_iso is not None else np.nan
}).describe()

mu_summary


In [ ]:
# Cell 18 — Quick plots (matplotlib)
import matplotlib.pyplot as plt

plt.figure(figsize=(7,4))
plt.hist(ak.to_numpy(mu_pt), bins=100, range=(0, 200), histtype="step")
plt.xlabel("Muon pT [GeV]")
plt.ylabel("Muons / bin")
plt.title("Muon pT (slice)")
plt.show()

plt.figure(figsize=(7,4))
plt.hist(ak.to_numpy(mu_eta), bins=60, range=(-3, 3), histtype="step")
plt.xlabel("Muon eta")
plt.ylabel("Muons / bin")
plt.title("Muon eta (slice)")
plt.show()


## Dimuon EDA (physics sanity check)

A recruiter/engineer will trust your pipeline more if you demonstrate a correct dimuon invariant mass spectrum.  
We build opposite-sign muon pairs and plot the mass distribution; you should see structures consistent with known resonances in collision data.


In [ ]:
# Cell 19 — Build dimuon pairs and invariant mass
MUON_MASS = 0.105658  # GeV

mu = vector.zip({
    "pt": arr["Muon_pt"],
    "eta": arr["Muon_eta"],
    "phi": arr["Muon_phi"],
    "mass": ak.ones_like(arr["Muon_pt"]) * MUON_MASS,
    "charge": arr["Muon_charge"],
})

# Basic quality selection (adjust after you inspect data):
# - at least mediumId OR tightId if available
qual = None
if "Muon_mediumId" in arr.fields:
    qual = (arr["Muon_mediumId"] == 1)
elif "Muon_tightId" in arr.fields:
    qual = (arr["Muon_tightId"] == 1)
else:
    qual = ak.ones_like(arr["Muon_pt"], dtype=bool)

mu_sel = mu[qual]

pairs = ak.combinations(mu_sel, 2, fields=["m1", "m2"])
os_pairs = pairs[(pairs.m1.charge * pairs.m2.charge) < 0]

dimu = os_pairs.m1 + os_pairs.m2
m_mumu = ak.flatten(dimu.mass)

len(m_mumu), ak.to_numpy(m_mumu[:10])


In [ ]:
# Cell 20 — Plot dimuon mass spectrum
plt.figure(figsize=(8,4))
plt.hist(ak.to_numpy(m_mumu), bins=200, range=(0, 200), histtype="step")
plt.xlabel("m(μ⁺μ⁻) [GeV]")
plt.ylabel("Pairs / bin")
plt.title("Opposite-sign dimuon invariant mass (slice)")
plt.yscale("log")
plt.show()

# Zoom around Z peak
plt.figure(figsize=(8,4))
plt.hist(ak.to_numpy(m_mumu), bins=120, range=(60, 120), histtype="step")
plt.xlabel("m(μ⁺μ⁻) [GeV]")
plt.ylabel("Pairs / bin")
plt.title("Zoom: Z region")
plt.show()


## Trigger inventory (HLT_) EDA

We’ll discover which `HLT_` branches exist in this file and then compute simple “fired fraction” estimates in our slice.


In [ ]:
# Cell 21 — Enumerate HLT branches (from this file)
hlt_branches = find_branches("HLT_", branch_names)
len(hlt_branches), hlt_branches[:40]


In [ ]:
# Cell 22 — Read a small subset of HLT branches (filter to muon-ish names)
# Keep only muon-related triggers to reduce I/O
hlt_mu = [b for b in hlt_branches if ("Mu" in b or "mu" in b)][:60]  # cap for speed

wanted2 = wanted + [b for b in hlt_mu if b not in wanted]
arr2 = events.arrays(wanted2, entry_stop=ENTRY_STOP, library="ak")

# Fired fraction table
hlt_rate = []
for b in hlt_mu:
    if b in arr2.fields:
        frac = float(ak.mean(arr2[b]))
        hlt_rate.append((b, frac))

hlt_rate_df = pd.DataFrame(hlt_rate, columns=["HLT_path", "fired_fraction_in_slice"]).sort_values(
    "fired_fraction_in_slice", ascending=False
)

hlt_rate_df.head(25)


## Scale EDA to multiple ROOT files

Now we repeat EDA across a few files to check stability (no single-file artifacts).  
We will:
- read a small number of events from each file
- compute summary metrics (nMuon mean, MET mean, dimuon-pair count, etc.)


In [ ]:
# Cell 23 — Multi-file EDA summarizer
def summarize_file(root_path, entry_stop=100_000):
    f = uproot.open(root_path)
    t = f["Events"]

    # Keep branches minimal
    branches = [b for b in ["run","luminosityBlock","PV_npvs","MET_pt","Muon_pt","Muon_eta","Muon_phi","Muon_charge","Muon_mediumId","Muon_tightId"] if b in t.keys()]
    a = t.arrays(branches, entry_stop=entry_stop, library="ak")

    nMuon = ak.num(a["Muon_pt"])
    mu_pt = ak.flatten(a["Muon_pt"])

    # Opposite-sign dimuons count (fast approximation)
    mu_charge = a["Muon_charge"]
    # Count OS pairs by brute-force combinations (ok for small entry_stop)
    mu_vec = vector.zip({
        "pt": a["Muon_pt"],
        "eta": a["Muon_eta"],
        "phi": a["Muon_phi"],
        "mass": ak.ones_like(a["Muon_pt"]) * MUON_MASS,
        "charge": mu_charge,
    })
    pairs = ak.combinations(mu_vec, 2, fields=["m1","m2"])
    os_pairs = pairs[(pairs.m1.charge * pairs.m2.charge) < 0]
    nOSPairs = ak.num(os_pairs.m1)

    out = {
        "file": root_path,
        "nEvents": len(a["run"]) if "run" in a.fields else len(a["Muon_pt"]),
        "run_min": int(ak.min(a["run"])) if "run" in a.fields else None,
        "run_max": int(ak.max(a["run"])) if "run" in a.fields else None,
        "nMuon_mean": float(ak.mean(nMuon)),
        "nMuon_p95": float(np.quantile(ak.to_numpy(nMuon), 0.95)),
        "mu_pt_mean": float(ak.mean(mu_pt)),
        "mu_pt_p95": float(np.quantile(ak.to_numpy(mu_pt), 0.95)),
        "MET_mean": float(ak.mean(a["MET_pt"])) if "MET_pt" in a.fields else np.nan,
        "nOSPairs_mean": float(ak.mean(nOSPairs)),
    }
    return out

summaries = []
for fp in tqdm(sample_files, desc="Summarizing files"):
    summaries.append(summarize_file(fp, entry_stop=80_000))

sum_df = pd.DataFrame(summaries)
sum_df


In [ ]:
# Cell 24 — Save EDA summary
sum_path = Path("/kaggle/working/eda_file_summaries.csv")
sum_df.to_csv(sum_path, index=False)
sum_path


## Next step after EDA (prepare for the project)

After you confirm:
- your dimuon mass spectrum looks sensible
- muon multiplicities are stable across files
- you understand which HLT paths exist and have non-trivial fired fractions

…then you can lock the project objective:
**“Explainable trigger emulation + drift monitoring”**:
- choose 3–6 HLT paths as labels
- build a Parquet training table from a validated run/lumi selection
- train an interpretable baseline + GBDT
- add drift detection over run/lumi blocks


In [ ]:
# Cell 25 — (Optional) Export a small Kaggle-ready Parquet slice for ML iteration
# This is NOT your final dataset, just a development slice.

out_parquet = Path("/kaggle/working/dev_slice.parquet")

# Flatten per-muon rows (common ML format). Keep run/lumi/event as keys.
# NOTE: this duplicates event-level info for each muon; that’s fine for quick iteration.
flat = ak.zip({
    "run": arr2["run"],
    "lumi": arr2["luminosityBlock"],
    "event": arr2["event"],
    "PV_npvs": arr2["PV_npvs"] if "PV_npvs" in arr2.fields else ak.zeros_like(arr2["run"]),
    "MET_pt": arr2["MET_pt"] if "MET_pt" in arr2.fields else ak.zeros_like(arr2["run"]),
    "Muon_pt": arr2["Muon_pt"],
    "Muon_eta": arr2["Muon_eta"],
    "Muon_phi": arr2["Muon_phi"],
    "Muon_charge": arr2["Muon_charge"],
    "Muon_mediumId": arr2["Muon_mediumId"] if "Muon_mediumId" in arr2.fields else ak.zeros_like(arr2["Muon_charge"]),
    "Muon_tightId": arr2["Muon_tightId"] if "Muon_tightId" in arr2.fields else ak.zeros_like(arr2["Muon_charge"]),
    "Muon_iso": arr2["Muon_pfRelIso03_all"] if "Muon_pfRelIso03_all" in arr2.fields else ak.zeros_like(arr2["Muon_pt"]),
})

flat_rows = ak.flatten(flat, axis=1)
df_flat = ak.to_dataframe(flat_rows).reset_index(drop=True)

# Add a couple of HLT labels (if present)
for b in hlt_mu[:10]:
    if b in arr2.fields:
        df_flat[b] = np.repeat(ak.to_numpy(arr2[b]), ak.to_numpy(ak.num(arr2["Muon_pt"])))

df_flat.to_parquet(out_parquet, index=False)
out_parquet, df_flat.head()
